##Functions

**Why use functions?**
   * Code reuse
   * Procedural decomposition
   * Alternative to cut-and-paste: redundancy

**Function topics**

   * The basics
   * Scope rules
   * Argument matching modes
   * Odds and ends
   * Generator expressions and functions
   * Design concepts
   * Functions are objects
   * Function gotchas

###Function basics

   * def is an executable statement; usually run during import
   * def creates a function object and assigns to a name
   * return sends a result object back to the caller
   * Arguments are passed by object reference (assignment)
   * Arguments, return types, and variables are not declared
   * Polymorphism: code to object interfaces, not datatypes

**General form**

```python
def <name>(arg1, arg2,… argN):
  <statements>
  return <value>
```

**Definition**

In [0]:
def times(x, y):      # create and assign function
  return x * y      # body executed when called

**Calls**

In [2]:
times(2, 4)           # arguments in parenthesis

8

In [4]:
times('Hi', 4)        # functions are 'typeless'

'HiHiHiHi'

“Polymorphism”


The meaning of an operation depends on its subject.
By not caring about types, code becomes more flexible.
Any object with a compatible interface will work.
Most errors are best caught by Python, not your code.

**Example: intersecting sequences**

   * Definition

In [0]:
def intersect(seq1, seq2):
  res = []                     # start empty
  for x in seq1:               # scan seq1
    if x in seq2:            # common item?
        res.append(x)        # add to end
  return res

* Calls

In [8]:
s1 = "SPAM"
s2 = "SCAM"
intersect(s1, s2)               # strings

['S', 'A', 'M']

In [9]:
intersect([1, 2, 3], (1, 4))    # mixed types

[1]

###Scope rules in functions

   * Enclosing module is a ‘global’ scope
   * Each call to a function is a new ‘local’ scope
   * Assigned names are local, unless declared “global”
   * All other names are global or builtin
   * Added in 2.2: ‘nonlocal’ enclosing function locals (if any) searched before global
   * Added in 3.X: ‘nonlocal’ variables can be changed if declared, just like ‘globals’


**Name resolution: the “LEGB” rule**

   *   References search up to 4 scopes:
1.    Local                                   (function)
2.    Enclosing functions           (if any)
3.    Global                                 (module)
4.    Builtin                                 (__builtin__ (2.X), builtins (3.X))
   *   Assignments create or change local names by default
   *   “global” declarations map assigned names to module

Example
   * Global names: ‘X’, ‘func’
   * Local names: ‘Y’, ‘Z’
   * Interactive prompt: module ‘__main__’

In [10]:
X = 99            # X and func assigned in module
def func(Y): # Y and Z assigned in function
  Z = X + Y     # X not assigned: global
  return Z
func(1) # func in module: result=100

100

**Enclosing Function Scopes (2.2+)**

In [12]:
def f1():
  x = 88
  def f2():
    print(x)         # 2.2: x found in enclosing function
  f2()

f1()

88


**More useful with lambda (ahead)**

In [15]:
def func():
  x = 42
  action = (lambda n: x ** n)          # 2.2
  return action

f = func()
f(2)

1764

**Most useful for closures: state retention (non-OOP)**

In [0]:
def maker(N):
  def action(X):            # make, don’t call
    return X ** N
  return action             # return new func

In [18]:
f = maker(3)                  # “remembers” 3 (N)
f(2), f(3)                    # arg to X, not N

(8, 27)

In [19]:
g = maker(4)                  # “remembers” 4 (N)
g(2), g(3)

(16, 81)

In [20]:
f(2), f(3)                    # f still has 3

(8, 27)

###More on “global”, and 3.X “nonlocal”

   * ‘global’ means assigned at top-level of a module file
   * Global names must be declared only if assigned
   * Global names may be referenced without being declared
   * 3.X: “nonlocal X” means X in enclosing def changeable

All Pythons

In [22]:
y, z = 1, 2         # global variables in module
def all_global():
  global x        # declare globals assigned
  x = y + z       # no need to declare y,z: 3-scope rule

all_global()  
print(x)  

3


enclosing function vars are changeable (state retention) (Python 3.X Only)

In [23]:
def outer():
  x = 1
  def inner():
    nonlocal x
    x += 1
    print(x)
  return inner

f = outer()      # f is really an inner
f()

2


In [24]:
f()

3


###More on “return”


   * Return sends back an object as value of call
   * Can return multiple arguments in a tuple
   * Can return modified argument name values

In [25]:
def multiple(x, y):
  x = 2
  y = [3, 4]
  return x, y

X = 1
L = [1, 2]
X, L = multiple(X, L)
X, L

(2, [3, 4])

###More on argument passing


   * Pass by object reference: assign shared object to local name
   * In def, assigning to argument name doesn’t effect caller
   * In def, changing mutable object argument may impact caller
   * Not pass ‘by reference’ (C++), but:

In [0]:
- Immutables act like ‘by value’ (C)
- Mutables act like ‘by pointer’ (C)

In [0]:
def changer(a, b):
...    a = 2             # changes local name's value only
...    b[0] = 'spam'     # changes shared object in-place


In [0]:
X = 1
L = [1, 2]
changer(X, L)
X, L

(1, ['spam', 2])
Equivalent to these assignments:

In [0]:
X = 1

In [0]:
a = X           # they share the same object

In [0]:
a = 2           # resets a only, X is still 1

In [0]:
L = [1, 2]

In [0]:
b = L           # they share the same object

In [0]:
b[0] = 'spam'   # in-place change: L sees the change too

Special argument matching modes
   * Positional      matched left-to-right in header (normal)
   * Keywords      matched by name in header
   * Varargs        catch unmatched positional or keyword args
   * Defaults        header can provide default argument values
   * 3.X: Keyword Only, after “*” in def,  must pass by name
Operation
Location
Interpretation
func(value)
caller
normal argument: matched by position
func(name=value)
caller
keyword argument: matched by name
def func(name)
function
normal argument: matches any by name or position
def func(name=value)
function
default argument value, if not passed in call
def func(*name)
function
matches remaining positional args (tuple)
def func(**name)
function
matches remaining keyword args (dictionary)
func(*args, **kargs)
caller
subsumes old apply(): unpack tuple/dict of args
def func(a, *b, c)
def func(a, *, c)
function
3.X keyword-only (c must be passed by name only)
About the stars…
In Python 3.4 and earlier, the special *X and **X star syntax forms can appear in 3 places:
1)     In assignments, where a *X in the recipient collects unmatched items in a new list (3.X sequence assignments)
2)     In function headers, where the two forms collect unmatched positional and keyword arguments in a tuple and dict
3)     In function calls, where the two forms unpack iterables and dictionaries into individual items (arguments)
In Python 3.5 and later, this star syntax is also usable within data structure literals—where it unpacks collections into individual items, like its original use in function calls (#3 above). The unpacking star syntax now also works in lists, tuples, sets, and dictionaries where it unpacks or "flattens" another object's contents in-place:

In [0]:
[x, *iter]      # list:  unpack iter's items
(x, *iter, y)   # tuple: ditto (parenthesis or not)
{*iter, x}      # set:   ditto (unordered, unique)
{x:y, **dict}   # dict:  unpack dict's keys/values)

For example, in 3.5+:

In [0]:
```python
x = [1, 2]
```
```python
y = [*x, *x]
```
```python
y
```
   [1, 2, 1, 2]

Examples
Positionals and keywords

In [0]:
def f(a, b, c): print a, b, c

In [0]:
f(1, 2, 3)

1 2 3

In [0]:
f(c=3, b=2, a=1)

1 2 3

In [0]:
f(1, c=3, b=2)

1 2 3
Defaults

In [0]:
def f(a, b=2, c=3): print a, b, c

In [0]:
f(1)

1 2 3

In [0]:
f(1, 4, 5)

1 4 5

In [0]:
f(1, c=6)

1 2 6
Arbitrary positionals

In [0]:
def f(*args): print args

In [0]:
f(1)

(1,)

In [0]:
f(1,2,3,4)

(1, 2, 3, 4)
Arbitrary keywords

In [0]:
def f(**args): print args

In [0]:
f()

{}

In [0]:
f(a=1, b=2)

{'a': 1, 'b': 2}

In [0]:
def f(a, *pargs, **kargs): print a, pargs, kargs

In [0]:
f(1, 2, 3, x=1, y=2)

1 (2, 3) {'y': 2, 'x': 1}
Example: min value functions
Example
   *   Only deals with matching: still passed by assignment
   *   Defaults retain an object: may change if mutable
def func(spam, eggs, toast=0, ham=0):   # first 2 required

In [0]:
print (spam, eggs, toast, ham)

func(1, 2)                      # output: (1, 2, 0, 0)
func(1, ham=1, eggs=0)          # output: (1, 0, 0, 1)
func(spam=1, eggs=0)            # output: (1, 0, 0, 0)
func(toast=1, eggs=2, spam=3)   # output: (3, 2, 1, 0)
func(1, 2, 3, 4)                # output: (1, 2, 3, 4)
Ordering rules
   * Call: keyword arguments after non-keyword arguments
   * Header: normals, then defaults, then *name, then **name
Matching algorithm (see exercise)
1.   Assign non-keyword arguments by position
2.   Assign keyword arguments by matching names
3.   Assign extra non-keyword arguments to *name tuple
4.   Assign extra keyword arguments to **name dictionary
5.   Unassigned arguments in header assigned default values
Odds and ends
   * lambda expression creates anonymous functions
   * list comprehensions, map, filter apply expressions to sequences (see also prior unit)
   * Generator expressions (2.4+)
   * Generator functions and yield (new in 2.2, 2.3)
   * apply function calls functions with arguments tuple
   * Functions return ‘None’ if they don’t use a real ‘return’
   * Python 3.X function annotations and keyword arguments
■    Lambda expressions

In [0]:
def func(x, y, z): return x + y + z

...

In [0]:
func(2, 3, 4)

9

In [0]:
f = lambda x, y, z: x + y + z

In [0]:
f(2, 3, 4)

9
hint: embedding logic in a lambda body
(A and B) or C
((A and [B]) or [C])[0]        # or newer: B if A else C
hint: embedding loops in a lambda body...next topic
■    List comprehensions (added in 2.0)
(See also prior unit)

In [0]:
ord('s')

115

In [0]:
res = []

In [0]:
for x in 'spam': res.append(ord(x))

...

In [0]:
res

[115, 112, 97, 109]

In [0]:
map(ord, 'spam')              # apply func to sequence

[115, 112, 97, 109]

In [0]:
[ord(x) for x in 'spam']      # apply expr to sequence

[115, 112, 97, 109]
# adding arbitrary expressions

In [0]:
[x ** 2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [0]:
map((lambda x: x**2), range(10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [0]:
lines = [line[:-1] for line in open('README.txt')]

In [0]:
lines[:2]

['This is Python version 2.4 alpha 3', '==================================']
# adding if tests

In [0]:
[x for x in range(10) if x % 2 == 0]

[0, 2, 4, 6, 8]

In [0]:
filter((lambda x: x % 2 == 0), range(10))

[0, 2, 4, 6, 8]
# advanced usage

In [0]:
[x**2 for x in range(10) if x % 2 == 0]

[0, 4, 16, 36, 64]

In [0]:
[x+y for x in 'abc' for y in 'lmn']

['al', 'am', 'an', 'bl', 'bm', 'bn', 'cl', 'cm', 'cn']

In [0]:
res = []

In [0]:
for x in 'abc':
...    for y in 'lmn':
...       res.append(x+y)
...

In [0]:
res

['al', 'am', 'an', 'bl', 'bm', 'bn', 'cl', 'cm', 'cn']
# nice for matrixes

In [0]:
M = [[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]

In [0]:
M[1]

[4, 5, 6]

In [0]:
col2 = [row[1] for row in M]

In [0]:
col2

[2, 5, 8]

In [0]:
quad = [M[i][j] for i in (0,1) for j in (0, 1)]

In [0]:
quad

[1, 2, 4, 5]
List comprehensions can become incomprehensible when nested, but map and list comprehensions may be faster than simple for loops
(In 2.4, comprehensions are twice as fast as for loops, and for loops are now quicker than map: see middle of this page, and CD’s Extras\Code\Misc\timerseqs.py)
■    Generator expressions (2.4+)
# list comprehensions generate entire list in memory

In [0]:
squares = [x**2 for x in range(5)]

In [0]:
squares

[0, 1, 4, 9, 16]
# generator expressions yield 1 result at a time: saves memory, distributes work

In [0]:
squares = (x**2 for x in range(5))

In [0]:
squares

<generator object at 0x00B2EC88>

In [0]:
next(squares)

0

In [0]:
next(squares)

1

In [0]:
next(squares)

4

In [0]:
list(squares)

[9, 16]
# iteration contexts automatically call next()

In [0]:
for x in (x**2 for x in range(5)):
        print x,
```    
0 1 4 9 16
```python
sum(x**2 for x in range(5))

30
■    Generator functions and yield
Generators implement iteration protocol: .next() (3.X .__next__())
Retains local scope when suspended
Distributes work over time, may save memory (see also: threads)
Related: generator expressions, enumerate function, file iterators
# functions compiled specially when contain yield

In [0]:
def gensquares(N):
...     for i in range(N):         # suspends and resumes itself
...         yield i ** 2           # <- return value and resume here later
 ```
# generator objects support iteration protocol: next()
```python
x = gensquares(10)

In [0]:
x                              # also retain all local variables between calls

<generator object at 0x0086C378>   # classes define __iter__ to return iter object

In [0]:
next(x)

0

In [0]:
next(x)

1

In [0]:
next(x)

4
…

In [0]:
next(x)

…StopIteration exception raised at end…
# for loops (and others) automatically call next()

In [0]:
for i in gensquares(5):        # resume the function each time
...     print i, ':',              # print last yielded value
...

0 : 1 : 4 : 9 : 16 :
# (advanced) application to coroutines via task switchers
►See Extras\Code\Misc\asynch1.py
# (advanced) generators, through the years…
►See Extras\Code\Misc\gendemo.py
Original
def with yield statement supports iteration protocol, returning yielded value for next()
In 2.5+
caller can resume generator and pass it value via G.send(X): result of “yield” expression
In 3.3+
“return” value allowed in generators: attached to exception, result of “yield from” expression
In 3.5+
“async” and “await” distinguish some coroutine roles, similar to “yield from”
More on generators and coroutines:
Coroutines in Python have morphed repeatedly—from yield, to 2.5’s yield/send, to 3.3’s yield from, and to 3.5’s async/await.  The latter of these tries to cast coroutines as distinct from generators, yet describes them as specialized generators, and provides linkage between new (“native”) and prior (“generator-based”) coroutines.  Either way, coroutines are an application-level topic—they’re just one asynchronous scheme in the concurrent programming domain, which also includes arguably more general threads and processes.  Because they are also of limited interest, we’ll punt here; see other resources and this review for more details on this obscure (and volatile!) corner of the Python language.
■    Apply built-in (2.X only) and syntax (all Pys)

In [0]:
def func(a, b, c):
        return a + b + c

In [0]:
apply(func, (2, 3, 4))         # 2.X only

9

In [0]:
apply(func, (2, 3), {'c': 4})

9
Versus Python 2.0+ apply-like call syntax
func(*a)        …like…   apply(func, a)
func(*a, **b)   …like…   apply(func, a, b)

In [0]:
func(*(2, 3, 4))

9

In [0]:
func(*(2, 3), **{'c': 4})     # 2.X and 3.X

9
# call syntax is more flexible:

In [0]:
def func(a, b, c, d): return a + b + c + d

In [0]:
args1 = (1, 2)

In [0]:
args2 = {'c': 3, 'd': 4}

In [0]:
func(*args1, **args2)

10

In [0]:
func(1, *(2,), **args2)

10
See also: “About the stars” sidebar above
■    Default return values

In [0]:
def proc(x):
...     print x

...

In [0]:
x = proc('testing 123...')

testing 123...

In [0]:
print x

None
■    Python 3.X function annotations

In [0]:
def func(a: int, b: 'spam', c: 88 = 99) -> float:
...     print(a, b, c)
...

In [0]:
func(1, 2)
1 2 99
```python
func.__annotations__

{'b': 'spam', 'return': <class 'float'>, 'a': <class 'int'>, 'c': 88}
■    Python 3.X keyword-only arguments

In [0]:
def f(a, b, *, c=3, d): print(a, b, c, d)

...

In [0]:
f(1, 2)

TypeError: f() missing 1 required keyword-only argument: 'd'

In [0]:
f(1, 2, 3)

TypeError: f() takes 2 positional arguments but 3 were given

In [0]:
f(1, 2, d=4)

1 2 3 4

In [0]:
f(1, 2, c=3, d=4)

1 2 3 4
Function design concepts
   * Use global variables only when absolutely necessary
   * Use arguments for input, ‘return’ for outputs
   * Don’t change mutable arguments unless expected
   * But globals are only state-retention tool without classes
   * But classes depend on mutable arguments (‘self’)
Functions are objects: indirect calls
   * Function objects can be assigned, passed, etc.
   * Can call objects generically: function, bound method, ...
def echo(message): print message
x = echo
x('Hello world!')
def indirect(func, arg):

In [0]:
func(arg)

indirect(echo, 'Hello world!')
schedule = [ (echo, 'Hello!'), (echo, 'Ni!') ]
for (func, arg) in schedule:

In [0]:
func(arg)

File scanners
file: scanfile.py
def scanner(name, function):

In [0]:
file = open(name, 'r')          # create file
for line in file.readlines():
    function(line)              # call function
file.close()

file: commands.py
import string
from scanfile import scanner
def processLine(line):

In [0]:
print string.upper(line)

scanner("data.txt", processLine)    # start scanner
Function gotchas
Local names are detected statically
# cant use both local+global version of name, unless reference through module

In [0]:
X = 99

In [0]:
def selector():        # X used but not assigned
...     print X            # X found in global scope
...

In [0]:
selector()

99

In [0]:
def selector():
...     print X            # does not yet exist!
...     X = 88             # X classified as a local name

...

In [0]:
selector()

Traceback (innermost last):
  File "<stdin>", line 1, in ?
  File "<stdin>", line 2, in selector
NameError: X

In [0]:
def selector():
...     global X           # force X to be global
...     print X
...     X = 88
...

In [0]:
selector()
99
Mutable defaults created just once
# to avoid: check for None inside def and set to [] if so
```python
def grow(A, B=[]):
        B.append(A)
        return B

In [0]:
grow(1)

[1]

In [0]:
grow(1)

[1, 1]

In [0]:
grow(1)

[1, 1, 1]
Nested functions weren’t nested scopes
# this works as of 2.2 – enclosing function scopes!

In [0]:
def outer(x):
...     def inner(i):          # assign in outer’s local
...         print i,           # i is in inner’s local
...         if i: inner(i-1)   # not in my local or global!
...     inner(x)

...

In [0]:
outer(3)

3
Traceback (innermost last):
  File "<stdin>", line 1, in ?
  File "<stdin>", line 6, in outer
  File "<stdin>", line 5, in inner
NameError: inner

In [0]:
def outer(x):
...     global inner
...     def inner(i):          # assign in enclosing module
...         print i,
...         if i: inner(i-1)   # found in my global scope
...     inner(x)

...

In [0]:
outer(3)

3 2 1 0
Use defaults to save references
# normally no longer necessary as of 2.2: enclosing function scopes
# BUT still required to retain current value of loop variables!

In [0]:
def outer(x, y):
        def inner():
            return x ** y
        return inner

In [0]:
x = outer(2, 4)

In [0]:
x()

16
# code before 2.2

In [0]:
def outer(x, y):
...     def inner(a=x, b=y):     # save x,y bindings/objects
...         return a**b          # from the enclosing scope
...     return inner

...

In [0]:
x = outer(2, 4)

In [0]:
x()

16

In [0]:
def outer(x, y):
...     return lambda a=x, b=y: a**b
...

In [0]:
y = outer(2, 5)
```python
y()

32
for I in someiterable:

In [0]:
actions.append(lambda I=I: …)  # retain current I, not last!

Optional reading: set functions
   * Functions process passed-in sequence objects
   * Work on any type of sequence objects
   * Supports mixed types: list and tuple, etc.
file: inter.py
def intersect(seq1, seq2):

In [0]:
res = []                     # start with an empty list
for x in seq1:               # scan the first sequence
    if x in seq2:
        res.append(x)        # add common items to end
return res

def union(seq1, seq2):

In [0]:
res = map(None, seq1)        # copy of seq1 (see ch13)
for x in seq2:               # add new items in seq2
    if not x in res:
        res.append(x)
return res

% python

In [0]:
from inter import intersect, union

In [0]:
s1 = "SPAM"

In [0]:
s2 = "SCAM"

In [0]:
intersect(s1, s2), union(s1, s2)           # strings

(['S', 'A', 'M'], ['S', 'P', 'A', 'M', 'C'])

In [0]:
intersect([1,2,3], (1,4))                  # mixed types

[1]

In [0]:
union([1,2,3], (1,4))

[1, 2, 3, 4]
Supporting multiple operands: *varargs
  
file: inter2.py
def intersect(*args):

In [0]:
res = []
for x in args[0]:                  # scan first sequence
    for other in args[1:]:         # for all other args
        if x not in other: break   # this in each one?
    else:
        res.append(x)              # add items to end
return res

def union(*args):

In [0]:
res = []
for seq in args:                   # for all args
    for x in seq:                  # for all nodes
        if not x in res:
            res.append(x)          # add items to result
return res

% python

In [0]:
from inter2 import intersect, union

In [0]:
s1, s2, s3 = "SPAM", "SCAM", "SLAM"

In [0]:
intersect(s1, s2), union(s1, s2)           # 2 operands

(['S', 'A', 'M'], ['S', 'P', 'A', 'M', 'C'])

In [0]:
intersect([1,2,3], (1,4))

[1]

In [0]:
intersect(s1, s2, s3)                      # 3 operands

['S', 'A', 'M']

In [0]:
union(s1, s2, s3)

['S', 'P', 'A', 'M', 'C', 'L']